# Question 1
Suppose that the data for analysis includes the attribute age. The age values for the
data tuples are (in increasing order) 13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25,
30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70. </br>
(a) Use min-max normalization to transform the values of age to the range[0:1]. </br>
(b) Use z-score normalization to transform the values of age. </br>
(c) Use normalization by decimal scaling to transform the values of age such that the transformed
value is less than 1.

In [ ]:
from scipy.stats import zscore
import pandas as pd


from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))


# Create a sample df
df = pd.DataFrame({'Age': [13, 15, 16, 16, 19, 20, 20, 21, 22, 22, 25, 25, 25, 25, 30, 33, 33, 35, 35, 35, 35, 36, 40, 45, 46, 52, 70]})

# Calculate the min-max and drop min-max into new column
df['min-max'] = (df['Age'] - df['Age'].min()) / (df['Age'].max() - df['Age'].min())    

# Calculate the zscores and drop zscores into new column
df['z_score'] = zscore(df['Age'])

# Calculate the decimal scaling and drop decimal scaling into new column
p = df['Age'].max()
q = len(str(abs(p)))
df['decimal_scaling'] = df['Age']/10**q

display(df)

<IPython.core.display.Javascript object>

,Age,min-max,z_score,decimal_scaling
0,13,0.000000,-1.335646,0.13
1,15,0.035088,-1.178168,0.15
2,16,0.052632,-1.099429,0.16
3,16,0.052632,-1.099429,0.16
4,19,0.105263,-0.863212,0.19
5,20,0.122807,-0.784473,0.20
6,20,0.122807,-0.784473,0.20
7,21,0.140351,-0.705734,0.21
8,22,0.157895,-0.626995,0.22
9,22,0.157895,-0.626995,0.22


# Question 2
Use the given dataset and perform the operations listed below.
Dataset Description
Attributes in the dataset:
  * Date - The date of the observation
  * Average Price - the average price of a single avocado
  * type - conventional or organic
  * year - the year
  * Region - the city or region of the observation
  * Total Volume - Total number of avocados sold
  * 4046 - Total number of avocados with PLU* 4046 sold
  * 4225 - Total number of avocados with PLU* 4225 sold
  * 4770 - Total number of avocados with PLU* 4770 sold

</br>(Product Lookup codes (PLU’s)) *




a. Sort the attribute “Total Volume” in the given dataset and distribute the data into equal
sized/frequency bins. Let the number of bins be 250. Smooth the sorted data by
* (i)bin-means 
* (ii) bin-medians 
* (iii) bin-boundaries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import plotly.express as px

from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

pd.set_option('display.expand_frame_repr', False) 

df = pd.read_csv('Avocado_Dataset.csv')
df_sorted = df.sort_values(by='Total Volume', ascending=True).reset_index()
df_sorted['Total Volume'] = df_sorted['Total Volume'].astype(int)


# Binning with equal frequency
qc, bins = pd.qcut(df_sorted['Total Volume'], q=250, precision=0, retbins=True)
print("Binning with equal frequency...........")
print(qc.value_counts())
print()
print(bins)
print("\n\n\n\n")
# print([i for i in qc])


# Binning with equal width
s_cut, bins = pd.cut(df_sorted['Total Volume'], 250, retbins=True)       # With retbins=True, we can get bins, i.e., a list of boundary values in addition to the binned data.
print("Binning with equal width...........")
print(s_cut.value_counts())
print()
print(bins)

# Smooth the sorted data by bean means
df1 = pd.DataFrame()
df1['Total Volume'] = df_sorted['Total Volume']
df1['bin'] = pd.cut(df1['Total Volume'], 250, include_lowest=True, right=False)
df1['mean'] = df1.groupby('bin')['Total Volume'].transform('mean')
del df1['Total Volume']
# df1.reset_index(drop=True)
df1 = df1.drop_duplicates()
print()
print(df1)


# Smooth the sorted data by bean median
df1 = pd.DataFrame()
df1['Total Volume'] = df_sorted['Total Volume']
df1['bin'] = pd.cut(df1['Total Volume'], 250, include_lowest=True, right=False)
df1['median'] = df1.groupby('bin')['Total Volume'].transform('median')
del df1['Total Volume']
# df1.reset_index(drop=True)
df1 = df1.drop_duplicates()
print()
print(df1)


# Smooth the sorted data by bean boundary
def boundry_settle(quantile, value):
  a, b = quantile.split(", ")
  a = float(a.strip("("))
  b = float(b.strip("]"))
  if value - a < b - value:
    return a
  else:
    return b

df1 = pd.DataFrame()
df1['Total Volume'] = df_sorted['Total Volume']
df1['bin'] = pd.qcut(df1['Total Volume'], 250).astype(str)
df1['bin_boundary'] = df1.apply(lambda row: boundry_settle(row["bin"], row["Total Volume"]), axis = 1)
del df1['Total Volume']
df1 = df1.drop_duplicates()
print()
print(df1)


<IPython.core.display.Javascript object>

Binning with equal frequency...........
(125592.0, 128166.0]    74
(2570.0, 2700.0]        74
(497040.0, 509215.0]    74
(16167.0, 16746.0]      74
(10497.0, 10726.0]      74
                        ..
(8933.0, 9146.0]        72
(35304.0, 37425.0]      72
(10726.0, 10967.0]      72
(509215.0, 523631.0]    72
(128166.0, 131634.0]    72
Name: Total Volume, Length: 250, dtype: int64

[8.40000000e+01 9.35992000e+02 1.11799200e+03 1.23300000e+03
 1.36498400e+03 1.49088000e+03 1.62490400e+03 1.73900000e+03
 1.83496800e+03 1.95596400e+03 2.08192000e+03 2.16895600e+03
 2.29190400e+03 2.42794800e+03 2.56988800e+03 2.70000000e+03
 2.81900000e+03 2.93993200e+03 3.04600000e+03 3.16477200e+03
 3.29900000e+03 3.42291600e+03 3.52891200e+03 3.62790800e+03
 3.77500000e+03 3.89680000e+03 4.03968800e+03 4.14878400e+03
 4.27288800e+03 4.41965200e+03 4.59188000e+03 4.73762800e+03
 4.90400000e+03 5.07600000e+03 5.25759200e+03 5.41730000e+03
 5.58085600e+03 5.74400000e+03 5.90284800e+03 6.09053200e+03
 6.334

b. The dataset represents weekly retail scan data for National retail volume (units) and price. However,
the company is interested in knowing the monthly (total per month) and annual sales (total per year),
rather than the total per week. So, reduce the data accordingly.

In [ ]:
import pandas as pd

from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

pd.set_option('display.expand_frame_repr', False)         # to prevent breaking of columns

df = pd.read_csv("Avocado_Dataset.csv", parse_dates =["Date"], index_col ="Date")
monthly_resampled_data = df.resample('M').mean()
print(monthly_resampled_data)
print('\n\n\n\n\n\n')


yearly_resampled_data = df.resample('12M').mean()
print(yearly_resampled_data)

<IPython.core.display.Javascript object>

            Total Volume           4046           4225          4770     Total Bags     Small Bags     Large Bags  XLarge Bags    year
Date                                                                                                                                  
2015-01-31  8.303570e+05  328527.572370  347458.252870  27078.729759  127292.472556  102345.971667   24091.092000   855.408889  2015.0
2015-02-28  7.359018e+05  308849.146914  279235.986173  22816.384228  125000.295401  107378.810340   17190.307778   431.177284  2015.0
2015-03-31  8.325426e+05  352764.705903  323913.518773  24039.963866  131824.391806  111019.566806   20196.701481   608.123519  2015.0
2015-04-30  7.730750e+05  300581.209282  319079.063796  24492.656713  128922.029444  104686.260718   23443.995949   791.772778  2015.0
2015-05-31  8.517748e+05  372357.014611  307569.349481  24876.348611  146972.093944  126208.858815   20128.942889   634.292241  2015.0
2015-06-30  8.380272e+05  327336.630853  323108.449852 

c. Summarize the number of missing values for each attribute

In [ ]:
import pandas as pd

from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

df = pd.read_csv("Avocado_Dataset.csv", parse_dates =["Date"], index_col ="Date")

print(df.isna().sum())

<IPython.core.display.Javascript object>

AveragePrice    30
Total Volume     0
4046             0
4225             0
4770             0
Total Bags       0
Small Bags       0
Large Bags       0
XLarge Bags      0
type             0
year             0
region           0
dtype: int64


d. Populate data for the missing values of the attribute= “Average Price” by averaging all the values of
the “Avg Price” attribute that fall under the same “REGION” attribute value.

In [ ]:
import pandas as pd
import numpy as np


from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

df = pd.read_csv("Avocado_Dataset.csv")

region_avg_price = df.groupby("region").mean("AveragePrice")
# df['AveragePrice'] = df['AveragePrice'].fillna(df.groupby('region')['AveragePrice'].transform('mean'))

df["AveragePrice"] = df.apply(
    lambda row: region_avg_price.loc[row["region"]] 
    if not row["AveragePrice"] 
    else row["AveragePrice"],
    axis=1
  )

print(df)

<IPython.core.display.Javascript object>

             Date AveragePrice  Total Volume     4046       4225    4770  Total Bags  Small Bags  Large Bags  XLarge Bags          type  year            region
0      27-12-2015         1.33      64236.62  1036.74   54454.85   48.16     8696.87     8603.62       93.25          0.0  conventional  2015            Albany
1      20-12-2015         1.35      54876.98   674.28   44638.81   58.33     9505.56     9408.07       97.49          0.0  conventional  2015            Albany
2      13-12-2015         0.93     118220.22   794.70  109149.67  130.50     8145.35     8042.21      103.14          0.0  conventional  2015            Albany
3      06-12-2015         1.08      78992.15  1132.00   71976.41   72.58     5811.16     5677.40      133.76          0.0  conventional  2015            Albany
4      29-11-2015         1.29      51039.60   941.48   43838.39   75.78     6183.95     5986.26      197.69          0.0  conventional  2015            Albany
...           ...          ...          

e. Discretize the attribute= “Date” using concept hierarchy into {Old, New, Recent} (Consider
2015,2016 : Old, 2017: New, 2018: Recent).

In [ ]:
import pandas as pd

from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

def classify_year(year):
  year = int(year)
  if year == 2015 or year == 2016:
    return "Old"
  elif year == 2017:
    return "New"
  elif year == 2018:
    return "Recent"
  else:
    return "Unknown"


df = pd.read_csv("Avocado_Dataset.csv")

df['Date'] =  pd.to_datetime(df['Date'], format='%d-%m-%Y')
df["Discrete_Date"] = df.apply(lambda row: classify_year(row["Date"].strftime("%Y")), axis=1)
print(df)

<IPython.core.display.Javascript object>

            Date AveragePrice  Total Volume     4046       4225    4770  Total Bags  Small Bags  Large Bags  XLarge Bags          type  year            region Discrete_Date
0     2015-12-27         1.33      64236.62  1036.74   54454.85   48.16     8696.87     8603.62       93.25          0.0  conventional  2015            Albany           Old
1     2015-12-20         1.35      54876.98   674.28   44638.81   58.33     9505.56     9408.07       97.49          0.0  conventional  2015            Albany           Old
2     2015-12-13         0.93     118220.22   794.70  109149.67  130.50     8145.35     8042.21      103.14          0.0  conventional  2015            Albany           Old
3     2015-12-06         1.08      78992.15  1132.00   71976.41   72.58     5811.16     5677.40      133.76          0.0  conventional  2015            Albany           Old
4     2015-11-29         1.29      51039.60   941.48   43838.39   75.78     6183.95     5986.26      197.69          0.0  conventional 